In [3]:
!pip install textstat --quiet
!pip install language_tool_python --quiet

In [51]:
import pandas as pd
from tqdm.notebook import tqdm

answers=pd.read_csv("data/final_results/llm_for_xai_results.csv")
answers

,student,explainer,strategy,evaluation_prompt,evaluation_answer1,question_0,question_1,question_2,question_3,question_4,...,question_6,question_7,question_8,yes_percentage,first_prompt,second_prompt,response1,response2,course,model
0,1,CEM,abnormal_conditions,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, NO, YES, YES]",1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,88.888889,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...","\n\nReport:\n\nBased on the model prediction, ...",\n\nHere is a shorter version of the report us...,dsp_001,meta-llama-3-70b-instruct
1,1,CEM,base_contrastive_explanation,Human: \n Based on the provided Input (if ...,"[YES, NO, YES, NO, YES, YES, YES, YES]",1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,NaN,75.000000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Understanding the Factors Contribu...,\n\nHere's a shorter version of the report usi...,dsp_001,meta-llama-3-70b-instruct
2,1,CEM,chain_of_thought,Human: \n Based on the provided Input (if ...,[],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nBased on the model prediction and the feat...,\n\nHere's a shorter version of the report usi...,dsp_001,meta-llama-3-70b-instruct
3,1,CEM,contrastive_explanation,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, NO, YES, YES, YES, YES, YES]",1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,88.888889,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...","\n\nReport:\n\nBased on the model prediction, ...",\n\nHere's a shorter version of the report usi...,dsp_001,meta-llama-3-70b-instruct
4,1,CEM,necessity_robustness_selection,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, YES]",1.0,1.0,1.0,1.0,1.0,...,1.0,NaN,NaN,100.000000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Insights into Student Behavior and...,\n\nHere's a shorter version of the report usi...,dsp_001,meta-llama-3-70b-instruct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21595,449,MC-LIME,contrastive_explanation,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, YES, YES, YES]",1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,100.000000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",## Contrastive Explanation for Student Perfor...,"Where Am I Going? You're doing well in your ""É...",geomatique_003,gemma2-9b-it
21596,449,MC-LIME,necessity_robustness_selection,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, NO]",1.0,1.0,1.0,1.0,1.0,...,0.0,NaN,NaN,85.714286,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...","## Analyzing Student Performance in ""Éléments...","You're doing well in ""Éléments de Géomatique""!...",geomatique_003,gemma2-9b-it
21597,449,MC-LIME,pearl_explanation,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, YES, YES]",1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,100.000000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",## Analysis of Student Performance: A Pearl's...,You're doing well in the course so far! Based...,geomatique_003,gemma2-9b-it
21598,449,MC-LIME,relevance_selection,Human: \n Based on the provided Input (if ...,"[YES, YES, YES, YES, YES, YES, YES, NO]",1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,NaN,87.500000,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",## Student Performance Analysis: Applying Rel...,You're doing well in the course so far! Based...,geomatique_003,gemma2-9b-it


In [10]:
test=answers.loc[:2].copy()

In [52]:
import textstat
import language_tool_python

# Initialize the language tool for grammar and spell check
tool = language_tool_python.LanguageTool('en-US')

# Define a function to calculate readability scores and grammar issues
def calculate_text_stats(text):
    flesch_kincaid = textstat.flesch_kincaid_grade(text)
    gunning_fog = textstat.gunning_fog(text)
    smog_index = textstat.smog_index(text)
    matches = tool.check(text)
    grammar_issues = len(matches)
    return flesch_kincaid, gunning_fog, smog_index, grammar_issues




In [64]:
# File path to the existing CSV file
output_file_path = "data/final_results/results_metrics_no_CoT.csv"

output_df=pd.read_csv("data/final_results/llm_for_xai_results.csv")

In [65]:
output_df.to_csv(output_file_path, index=False)

# Ensure that the columns exist; if not, create them with NaN values
for column in ['Flesch_Kincaid', 'Gunning_Fog', 'SMOG_Index', 'Grammar_Issues']:
    if column not in output_df.columns:
        output_df[column] = pd.NA  # Initialize the column with NaN values

In [67]:
# Step 2: Define the strings to be removed
strings_to_keep = [
    "[YES, YES, YES, YES, YES]",
    "[YES, YES, YES, NO, YES]",
    "[YES, YES, YES, YES, NO]",
    "[YES, YES, YES, NO, NO]"
]

# Step 2: Filter the DataFrame
# For rows where 'strategy' is 'chain_of_thought', only keep those with 'evaluation_answer1' in strings_to_keep
output_df = output_df[
    (output_df['strategy'] != "chain_of_thought") |
    (output_df['strategy'] == "chain_of_thought") & (output_df['model']=="gpt-4o")
]

In [68]:
output_df.to_csv(output_file_path, index=False)

In [69]:
# Apply tqdm to the DataFrame's 'apply' method
tqdm.pandas(desc="Calculating text stats")

# Apply the function to each row with progress tracking
for index, row in tqdm(output_df.iterrows(), total=len(output_df), desc="Processing rows"):

    if pd.notna(row['Flesch_Kincaid']) and pd.notna(row['Gunning_Fog']) and pd.notna(row['SMOG_Index']) and pd.notna(row['Grammar_Issues']):
        continue  # Skip this row if all values are not NaN

    # Calculate the text stats for the current row
    flesch_kincaid, gunning_fog, smog_index, grammar_issues = calculate_text_stats(row['response1'])
    
    # Update the existing DataFrame with the new columns
    output_df.loc[index, 'Flesch_Kincaid'] = flesch_kincaid
    output_df.loc[index, 'Gunning_Fog'] = gunning_fog
    output_df.loc[index, 'SMOG_Index'] = smog_index
    output_df.loc[index, 'Grammar_Issues'] = grammar_issues
    
    # Save the updated DataFrame back to the file after each update
    output_df.to_csv(output_file_path, index=False)

print("All rows processed and saved.")

Processing rows:   0%|          | 0/19800 [00:00<?, ?it/s]

All rows processed and saved.


In [15]:
# # Calculate the average for each explainer
# explainer_averages = answers.groupby('explainer')[['Flesch_Kincaid', 'Gunning_Fog', 'SMOG_Index']].mean().reset_index()

# # Calculate the average for each strategy
# strategy_averages = answers.groupby('strategy')[['Flesch_Kincaid', 'Gunning_Fog', 'SMOG_Index']].mean().reset_index()

# course_averages = answers.groupby('course')[['Flesch_Kincaid', 'Gunning_Fog', 'SMOG_Index']].mean().reset_index()

# model_averages = answers.groupby('model')[['Flesch_Kincaid', 'Gunning_Fog', 'SMOG_Index']].mean().reset_index()


In [16]:
# latex_table=explainer_averages.to_latex(index=False)
# latex_table

'\\begin{tabular}{lrrr}\n\\toprule\nexplainer & Flesch_Kincaid & Gunning_Fog & SMOG_Index \\\\\n\\midrule\nCEM & 13.466667 & 12.860000 & 14.766667 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [17]:
# strategy_averages

,strategy,Flesch_Kincaid,Gunning_Fog,SMOG_Index
0,abnormal_conditions,14.1,13.87,15.5
1,base_contrastive_explanation,12.4,11.79,14.1
2,chain_of_thought,13.9,12.92,14.7
